##### Task 3: Interactive Dashboard Development

In [ ]:
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.offline as pyo
from datetime import datetime, timedelta
import json

from data_loader import DataLoader
from event_manager import EventManager
from change_point_model import EnhancedChangePointModel

# Enable offline plotting
pyo.init_notebook_mode(connected=True)

%matplotlib inline

##### 1. Data Preparation for Dashboard

In [ ]:
# Load and prepare data
loader = DataLoader('../data/BrentOilPrices.csv')
df = loader.load_data()
event_manager = EventManager()
events = event_manager.get_all_events()

print(f"Data loaded: {len(df)} price records, {len(events)} events")

# Prepare data for dashboard
dashboard_data = {
    'prices': df[['Date', 'Price', 'Log_Returns']].copy(),
    'events': events.copy(),
    'summary_stats': loader.get_summary_stats(df)
}

# Convert dates to strings for JSON serialization
dashboard_data['prices']['Date'] = dashboard_data['prices']['Date'].dt.strftime('%Y-%m-%d')
dashboard_data['events']['date'] = dashboard_data['events']['date'].dt.strftime('%Y-%m-%d')

##### 2. Interactive Time Series Visualization

In [ ]:
def create_interactive_price_chart(df, events, title="Brent Oil Prices with Events"):
    """Create interactive price chart with event markers"""
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        subplot_titles=('Price (USD/barrel)', 'Log Returns'),
        row_heights=[0.7, 0.3]
    )
    
    # Add price trace
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df['Price'],
            mode='lines',
            name='Brent Oil Price',
            line=dict(color='blue', width=1),
            hovertemplate='Date: %{x}<br>Price: $%{y:.2f}<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Add log returns trace
    fig.add_trace(
        go.Scatter(
            x=df['Date'][1:],
            y=df['Log_Returns'].dropna(),
            mode='lines',
            name='Log Returns',
            line=dict(color='red', width=0.5),
            hovertemplate='Date: %{x}<br>Return: %{y:.4f}<extra></extra>'
        ),
        row=2, col=1
    )
    
    # Add event markers
    colors = {'Conflict': 'red', 'Economic': 'orange', 'OPEC': 'green', 
              'Geopolitical': 'purple', 'Natural Disaster': 'brown'}
    
    for _, event in events.iterrows():
        event_date = pd.to_datetime(event['date'])
        
        # Find corresponding price
        price_data = df[df['Date'] <= event_date]
        if len(price_data) > 0:
            price_at_event = price_data['Price'].iloc[-1]
            
            # Add vertical line for high impact events
            if event['impact'] == 'High':
                fig.add_vline(
                    x=event_date,
                    line_dash="dash",
                    line_color=colors.get(event['category'], 'gray'),
                    opacity=0.7,
                    annotation_text=event['event'][:30] + '...',
                    annotation_position="top"
                )
    
    # Update layout
    fig.update_layout(
        title=title,
        height=600,
        showlegend=True,
        hovermode='x unified'
    )
    
    fig.update_xaxes(title_text="Date", row=2, col=1)
    
    return fig

# Create and display interactive chart
interactive_fig = create_interactive_price_chart(df, events)
interactive_fig.show()

##### 3. Change Point Analysis Dashboard Component

In [ ]:
def create_change_point_dashboard(model, cp_summary):
    """Create change point analysis dashboard"""
    
    # Create subplots for different aspects
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Time Series with Change Point',
            'Change Point Posterior Distribution',
            'Parameter Comparison',
            'Model Diagnostics'
        ),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # 1. Time series with change point
    series_data = model.series
    tau_mean = cp_summary['mean_tau']
    
    fig.add_trace(
        go.Scatter(
            y=series_data,
            mode='lines',
            name='Data',
            line=dict(color='blue', width=1)
        ),
        row=1, col=1
    )
    
    fig.add_vline(
        x=tau_mean,
        line_dash="dash",
        line_color="red",
        row=1, col=1
    )
    
    # 2. Change point posterior
    tau_samples = cp_summary['tau_samples']
    hist_data = np.histogram(tau_samples, bins=50)
    
    fig.add_trace(
        go.Bar(
            x=hist_data[1][:-1],
            y=hist_data[0],
            name='Posterior',
            marker_color='lightblue'
        ),
        row=1, col=2
    )
    
    # 3. Parameter comparison (if model is fitted)
    if model.trace is not None:
        mu1_samples = model.trace.posterior['mu1'].values.flatten()
        mu2_samples = model.trace.posterior['mu2'].values.flatten()
        
        fig.add_trace(
            go.Histogram(
                x=mu1_samples,
                name='μ₁ (before)',
                opacity=0.7,
                nbinsx=30
            ),
            row=2, col=1
        )
        
        fig.add_trace(
            go.Histogram(
                x=mu2_samples,
                name='μ₂ (after)',
                opacity=0.7,
                nbinsx=30
            ),
            row=2, col=1
        )
    
    # 4. Model diagnostics placeholder
    fig.add_trace(
        go.Scatter(
            x=[1, 2, 3],
            y=[1, 1, 1],
            mode='markers+text',
            text=['R-hat', 'ESS', 'Convergence'],
            textposition="middle center",
            name='Diagnostics'
        ),
        row=2, col=2
    )
    
    fig.update_layout(
        title="Change Point Analysis Dashboard",
        height=800,
        showlegend=True
    )
    
    return fig

# Example usage (would need fitted model)
print("Change point dashboard component created.")
print("This would be populated with actual model results in the full dashboard.")

##### 4. Event Analysis Dashboard

In [ ]:
def create_event_analysis_dashboard(events):
    """Create event analysis dashboard"""
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Events by Category',
            'Events by Impact Level',
            'Events Timeline',
            'Recent High-Impact Events'
        ),
        specs=[[{"type": "pie"}, {"type": "pie"}],
               [{"colspan": 2}, None]],
        vertical_spacing=0.1
    )
    
    # 1. Events by category
    category_counts = events['category'].value_counts()
    fig.add_trace(
        go.Pie(
            labels=category_counts.index,
            values=category_counts.values,
            name="Category"
        ),
        row=1, col=1
    )
    
    # 2. Events by impact
    impact_counts = events['impact'].value_counts()
    fig.add_trace(
        go.Pie(
            labels=impact_counts.index,
            values=impact_counts.values,
            name="Impact"
        ),
        row=1, col=2
    )
    
    # 3. Events timeline
    events_sorted = events.sort_values('date')
    colors = {'High': 'red', 'Medium': 'orange', 'Low': 'green'}
    
    for impact in ['High', 'Medium', 'Low']:
        impact_events = events_sorted[events_sorted['impact'] == impact]
        fig.add_trace(
            go.Scatter(
                x=pd.to_datetime(impact_events['date']),
                y=[impact] * len(impact_events),
                mode='markers',
                name=f'{impact} Impact',
                marker=dict(
                    color=colors[impact],
                    size=10,
                    symbol='circle'
                ),
                text=impact_events['event'],
                hovertemplate='%{text}<br>Date: %{x}<extra></extra>'
            ),
            row=2, col=1
        )
    
    fig.update_layout(
        title="Event Analysis Dashboard",
        height=700,
        showlegend=True
    )
    
    return fig

# Create and display event dashboard
event_fig = create_event_analysis_dashboard(events)
event_fig.show()

##### 5. Flask Backend API Structure

In [ ]:
# Flask backend API structure (would be in separate .py file)
flask_api_code = '''
from flask import Flask, jsonify, request
from flask_cors import CORS
import pandas as pd
import json
from datetime import datetime

from data_loader import DataLoader
from event_manager import EventManager
from enhanced_change_point_model import EnhancedChangePointModel

app = Flask(__name__)
CORS(app)

# Global data storage
loader = DataLoader('data/BrentOilPrices.csv')
df = loader.load_data()
event_manager = EventManager()

@app.route('/api/data/prices', methods=['GET'])
def get_prices():
    """Get historical price data"""
    start_date = request.args.get('start_date')
    end_date = request.args.get('end_date')
    
    data = df.copy()
    if start_date:
        data = data[data['Date'] >= start_date]
    if end_date:
        data = data[data['Date'] <= end_date]
    
    return jsonify({
        'data': data[['Date', 'Price', 'Log_Returns']].to_dict('records'),
        'count': len(data)
    })

@app.route('/api/events', methods=['GET'])
def get_events():
    """Get historical events"""
    category = request.args.get('category')
    impact = request.args.get('impact')
    
    events = event_manager.get_all_events()
    
    if category:
        events = events[events['category'] == category]
    if impact:
        events = events[events['impact'] == impact]
    
    return jsonify({
        'events': events.to_dict('records'),
        'count': len(events)
    })

@app.route('/api/analysis/changepoint', methods=['POST'])
def run_changepoint_analysis():
    """Run change point analysis"""
    params = request.json
    
    series_type = params.get('series_type', 'log_returns')
    n_samples = params.get('n_samples', 1000)
    start_date = params.get('start_date')
    end_date = params.get('end_date')
    
    # Filter data if date range specified
    data = df.copy()
    if start_date:
        data = data[data['Date'] >= start_date]
    if end_date:
        data = data[data['Date'] <= end_date]
    
    # Select series
    if series_type == 'log_returns':
        series = data['Log_Returns'].dropna()
        dates = data['Date'][1:]
    else:
        series = data['Price']
        dates = data['Date']
    
    try:
        # Fit model
        model = EnhancedChangePointModel(series, dates)
        trace = model.fit(n_samples=n_samples, tune=500)
        
        # Get results
        summary = model.get_change_point_summary()
        convergence = model.check_convergence()
        
        return jsonify({
            'success': True,
            'change_point': {
                'index': int(summary['mode_tau']),
                'date': summary.get('mode_date', '').strftime('%Y-%m-%d') if 'mode_date' in summary else None,
                'confidence_interval': summary['tau_95_hdi'].tolist()
            },
            'convergence': convergence['converged'],
            'parameters': model.get_parameter_summary().to_dict()
        })
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 500

@app.route('/api/analysis/events/associate', methods=['POST'])
def associate_events():
    """Associate change point with events"""
    params = request.json
    change_date = params.get('change_date')
    days_threshold = params.get('days_threshold', 60)
    
    if not change_date:
        return jsonify({'error': 'change_date required'}), 400
    
    nearby_events = event_manager.find_nearest_event(change_date, days_threshold)
    
    return jsonify({
        'events': nearby_events.to_dict('records'),
        'count': len(nearby_events)
    })

if __name__ == '__main__':
    app.run(debug=True, port=5000)
'''

print("Flask API structure defined.")
print("This would be saved as 'dashboard_api.py' for the backend server.")

##### 6. Dashboard Features Summary

In [ ]:
# Create a comprehensive dashboard features overview
dashboard_features = {
    "Interactive Visualizations": [
        "Time series plots with zoom and pan",
        "Event markers with hover information",
        "Change point detection results",
        "Parameter distribution plots",
        "Model diagnostics displays"
    ],
    "User Controls": [
        "Date range selection",
        "Series type selection (price/returns)",
        "Model parameters adjustment",
        "Event filtering by category/impact",
        "Analysis trigger buttons"
    ],
    "Real-time Analysis": [
        "On-demand change point detection",
        "Event association analysis",
        "Impact quantification",
        "Model convergence monitoring",
        "Results export functionality"
    ],
    "Business Intelligence": [
        "Executive summary reports",
        "Risk assessment metrics",
        "Investment implications",
        "Policy recommendations",
        "Market trend analysis"
    ]
}

print("DASHBOARD FEATURES OVERVIEW")
print("=" * 50)

for category, features in dashboard_features.items():
    print(f"\n{category}:")
    for feature in features:
        print(f"  • {feature}")

print("\n" + "=" * 50)

##### 7. Sample Dashboard Data Export

In [ ]:
# Export sample data for dashboard development
sample_data = {
    'metadata': {
        'title': 'Brent Oil Price Change Point Analysis Dashboard',
        'description': 'Interactive analysis of structural breaks in oil prices',
        'data_range': f"{df['Date'].min()} to {df['Date'].max()}",
        'total_records': len(df),
        'total_events': len(events)
    },
    'sample_prices': df.head(100)[['Date', 'Price', 'Log_Returns']].to_dict('records'),
    'sample_events': events.head(10).to_dict('records'),
    'event_categories': events['category'].value_counts().to_dict(),
    'impact_levels': events['impact'].value_counts().to_dict()
}

# Convert dates to strings for JSON serialization
for record in sample_data['sample_prices']:
    if pd.notna(record['Date']):
        record['Date'] = pd.to_datetime(record['Date']).strftime('%Y-%m-%d')

for record in sample_data['sample_events']:
    if pd.notna(record['date']):
        record['date'] = pd.to_datetime(record['date']).strftime('%Y-%m-%d')

# Save sample data
with open('../data/dashboard_sample_data.json', 'w') as f:
    json.dump(sample_data, f, indent=2, default=str)

print("Sample dashboard data exported to '../data/dashboard_sample_data.json'")
print(f"Data includes {len(sample_data['sample_prices'])} price records and {len(sample_data['sample_events'])} events")

### 9. Dashboard Implementation Roadmap

### Phase 1: Backend Development
1. **Flask API Setup**: Implement REST endpoints for data and analysis
2. **Data Processing**: Optimize data loading and caching
3. **Model Integration**: Connect PyMC3 models to API endpoints
4. **Error Handling**: Robust error handling and validation

### Phase 2: Frontend Development
1. **React Setup**: Initialize React application with routing
2. **Component Library**: Build reusable chart and UI components
3. **State Management**: Implement Redux for application state
4. **API Integration**: Connect frontend to Flask backend

### Phase 3: Advanced Features
1. **Real-time Updates**: WebSocket integration for live analysis
2. **User Authentication**: Secure access and user management
3. **Export Functionality**: PDF reports and data downloads
4. **Mobile Responsiveness**: Optimize for mobile devices

### Phase 4: Deployment
1. **Containerization**: Docker setup for easy deployment
2. **Cloud Deployment**: AWS/Azure deployment configuration
3. **Performance Optimization**: Caching and load balancing
4. **Monitoring**: Application monitoring and logging

## Key Deliverables for Task 3:

1. **Interactive Visualizations**: Plotly-based charts with user interaction
2. **Flask Backend API**: RESTful endpoints for data and analysis
3. **Dashboard Architecture**: Scalable frontend-backend separation
4. **Real-time Analysis**: On-demand change point detection
5. **Business Intelligence**: Executive reporting and insights

The dashboard provides stakeholders with:
- **Interactive exploration** of oil price data and events
- **Real-time analysis** capabilities for different time periods
- **Professional visualizations** suitable for presentations
- **Quantified insights** for investment and policy decisions